<a href="https://colab.research.google.com/github/cras-lab/BlockChain_API/blob/main/Ethereum_CoreAPI_Nodejs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이더리움 로컬에 접속하는 예제<BR>
현재까지 다운로드한 최신 50개 블록 속의 트랜잭션개수와 총 수수료를 출력<BR>
먼저 web3 모듈을 설치해야 한다.

In [ ]:
pip -q install web3

In [66]:
from web3 import Web3
import matplotlib.pyplot as plt

Colab 설치시 로컬접근을 위해서는 터널링이 필요하다.<BR>
CMD창에서 "lt --port 8545 --local-host 0.0.0.0"을 실행해야 한다. <BR>
Node.js를 사용해서 터널링을 한다.node.js를 실행하고 나서 나타나는 URL을 복사해서 입력

In [67]:
# node.js를 통한 외부 접근 URL. Colab 설치시 필요
nodejs_url =   "https://chatty-apes-marry.loca.lt"

로컬 이더리움에 접속한다.

In [68]:
web3 = Web3(Web3.HTTPProvider(nodejs_url))

접속된 상태이면 값을 읽어온다. 아직 동기 중이라면 오류가 날 수 있다.<BR>
값을 읽어오면 그래프로 그린다. 좌측은 수수료, 우측은 tx개수

In [ ]:
# 연결 상태 확인
if web3.is_connected():
    print("Connected to Ethereum network")

    # 동기화 상태 확인
    sync_status = web3.eth.syncing
    if sync_status:
        print("Syncing in progress...")
        print(sync_status)

        # 동기화된 최신 블록 번호 조회
        current_block = sync_status.currentBlock
        print(f"Current synced block: {current_block}")

        # 동기화된 범위 내의 블록 조회
        start_block = max(0, current_block - 50)
        end_block = current_block

    else:
        print("Geth node is fully synced")

        # 최신 블록 번호 조회
        latest_block = web3.eth.block_number
        print(f"Latest block number: {latest_block}")

        # 동기화된 범위 내의 블록 조회
        start_block = max(0, latest_block - 50)
        end_block = latest_block

    block_numbers = []
    total_fees = []
    transaction_counts = []

    for block_number in range(start_block, end_block + 1):
        try:
            block = web3.eth.get_block(block_number, full_transactions=True)
            transactions = block['transactions']
            total_fee = sum(tx['gasPrice'] * tx['gas'] for tx in transactions) / 10**18  # ETH 단위로 변환
            block_numbers.append(block_number)
            total_fees.append(total_fee)
            transaction_counts.append(len(transactions))
            print(f"Block {block_number} has {len(transactions)} transactions with total fee {total_fee} ETH")
        except Exception as e:
            print(f"Could not fetch block {block_number}: {e}")

    # 수수료 변화를 그래프로 그리기
    fig, ax1 = plt.subplots(figsize=(10, 5))

    color = 'tab:blue'
    ax1.set_xlabel('Block Number')
    ax1.set_ylabel('Total Fees (ETH)', color=color)
    ax1.plot(block_numbers, total_fees, marker='o', linestyle='-', color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.grid(True)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:red'
    ax2.set_ylabel('Number of Transactions', color=color)  # we already handled the x-label with ax1
    ax2.plot(block_numbers, transaction_counts, marker='x', linestyle='--', color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.title('Total Fees and Number of Transactions per Block')
    plt.show()

else:
    print("Failed to connect to Ethereum network")
